In [106]:
import datasets
from datasets import load_dataset
import pandas as pd
import nltk
import seaborn as sns
import os
import matplotlib.pyplot as plt
import json
import re
import numpy as np
from wordcloud import WordCloud

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set('indonesian') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rowjak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rowjak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
path = os.getcwd()

In [108]:
df_train = pd.read_parquet(os.path.join(path, "dataset/train_combined.parquet"))

In [109]:
df_train_sample = pd.read_parquet(os.path.join(path, "dataset/train_sample_combined.parquet"))
df_xtreme_dev = pd.read_parquet(os.path.join(path, "dataset/xtreme_dev_combined.parquet"))
df_xtreme_test = pd.read_parquet(os.path.join(path, "dataset/xtreme_test_combined.parquet"))

In [121]:
df_train_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38777 entries, 42855 to 90081
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  38777 non-null  int64 
 1   url                 38777 non-null  object
 2   clean_article       38777 non-null  object
 3   clean_summary       38777 non-null  object
 4   extractive_summary  38777 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [124]:
df_test = pd.read_parquet(os.path.join(path, "dataset/test_combined.parquet"))

In [146]:
def extract_article(sentences):
    # Gabungkan array kata-kata menjadi kalimat
    text = ' '.join([' '.join(sentence) for sentence in sentences])
    # Hilangkan spasi sebelum tanda baca seperti .,;:!?()
    text = re.sub(r'\s+([.,:;!?()])', r'\1', text)
    # Hilangkan spasi setelah tanda baca '.' jika huruf setelahnya kecil
    text = re.sub(r'\.\s+([a-z])', r'.\1', text)
    
    text = re.sub(r'\(\s+', '(', text)  # Hilangkan spasi setelah '('
    text = re.sub(r'\s+\)', ')', text)  # Hilangkan spasi sebelum ')'
    
    text = re.sub(r'\[\s+', '[', text)  # Hilangkan spasi setelah '['
    text = re.sub(r'\s+\]', ']', text)  # Hilangkan spasi sebelum ']'
    
    return text

In [155]:
df_used = df_test

In [156]:
df_used['ext_clean_article'] = df_used['clean_article'].apply(extract_article)
df_used['ext_clean_summary'] = df_used['clean_summary'].apply(extract_article)

In [157]:
def extract_sentences_from_index(clean_article, extractive_summary):
    return [clean_article[i] for i in extractive_summary]

In [158]:
df_used['extractive_summary_sentences'] = df_used.apply(lambda row: extract_sentences_from_index(row['clean_article'], row['extractive_summary']), axis=1)

In [159]:
df_used['ext_extractive_summary'] = df_used['extractive_summary_sentences'].apply(extract_article)

In [168]:
def preprocessing_article(text):
    # ubah semua menjadi huruf kecil
    text = text.lower()
    
    # hilangkan teks yang memiliki tanda kurun () dan []
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    
    # hilangkan awalan liputan6.com kemudian setelah itu tanda :
    text = re.sub(r'^liputan6\.com, [^:]+: ', '', text)
    
    # hilangkan semua tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    
    # hilangkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [171]:
df_used['prep_clean_article'] = df_used['ext_clean_article'].apply(preprocessing_article)
df_used['prep_clean_summary'] = df_used['ext_clean_summary'].apply(preprocessing_article)

In [172]:
random_sample = df_used.sample(n=10, random_state=1)  # random_state untuk konsistensi hasil
for index, row in random_sample.iterrows():
    print(f"1.Index: {index}")
    print(f"2.Url:\n {row['url']}")
    
    print(f"3.1.Clean Article:\n {row['ext_clean_article']}")
    print(f"3.2.Prep Clean Article:\n {row['prep_clean_article']}")
    
    print(f"4.1.Clean Summary:\n {row['ext_clean_summary']}")
    print(f"4.2.Prep Clean Summary:\n {row['prep_clean_summary']}")
    
    print(f"5.Extractive Summary:\n {row['ext_extractive_summary']}")
    print("-" * 100)  # Pembatas antar tweet

1.Index: 5959
2.Url:
 https://www.liputan6.com/news/read/20289/dihadang-massa--gedung-sekolah-korea-batal-dieksekusi
3.1.Clean Article:
 Liputan6.com, Jakarta: Petugas juru sita Pengadilan Negeri Jakarta Timur batal mengeksekusi Gedung Sekolah Korea di Kelurahan Ceger, Cipayung, Rabu(19/9). Pasalnya, pemilik sekolah tersebut menghalanginya dengan mengerahkan massa. Untuk menghindari bentrokan fisik, jajaran Kepolisian Sektor Cipayung menyarankan petugas eksekutor PN Jaktim mengurungkan niatnya. Kuasa hukum Sekolah Korea, Iqbal Salim mengatakan, kliennya mempunyai sertifikat sah atas tanah dan bangunan tersebut. Iqbal menjelaskan, tanah seluas delapan ribu meter persegi itu dibeli dari seorang di antara ahli warisnya. Namun belakangan hari, penjualan tanah tersebut mendapat gugatan dari ahli waris yang lainnya.(ANS/Gaby Getal dan Ary Trisna).
3.2.Prep Clean Article:
 petugas juru sita pengadilan negeri jakarta timur batal mengeksekusi gedung sekolah korea di kelurahan ceger cipayung rab

In [38]:
df_xtreme_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4948 entries, 0 to 4947
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4948 non-null   int64 
 1   url                 4948 non-null   object
 2   clean_article       4948 non-null   object
 3   clean_summary       4948 non-null   object
 4   extractive_summary  4948 non-null   object
dtypes: int64(1), object(4)
memory usage: 193.4+ KB


In [1]:
import pandas as pd
import os
import re
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import transformers
import torch

In [3]:
torch.cuda.is_available()

True